In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/docee/train_all.csv")
df.head()

,id,title,text,event_type,arguments,date,metadata
0,0,Vietnam reelects conservative Nguyễn Phú Trọng...,Vietnam's Communist Party Wednesday re-elected...,Government Job change - Election,"[{'start': 0, 'end': 24, 'type': 'Candidates a...",January 2016,"['(AP via ABC News)', '(Channel NewsAsia)']"
1,1,At least 42 people are killed in a bus crash i...,Another 43 people were injured when the bus ca...,Road Crash,"[{'start': 8, 'end': 29, 'type': 'Casualties a...",October 2006,['(BBC)']
2,2,At least 27 migrants die in a shipwreck in the...,At least 27 migrants have died off the Turkish...,Shipwreck,"[{'start': 0, 'end': 29, 'type': 'Casualties a...",February 2016,"['(ANSAmed)', '(Leadership)', '(news.com.au)',..."
3,3,Colten Treu faces charges of vehicular homicid...,"Colten Treu, 21, and his roommate both told au...",Road Crash,"[{'start': 183, 'end': 207, 'type': 'Number of...",November 2018,"['(KSTP)', '(Oxygen)']"
4,4,"Hours after the announcement, Morales resigns ...",Bolivian President Evo Morales has resigned af...,Government Job change - Resignation_Dismissal,"[{'start': 0, 'end': 17, 'type': 'Position', '...",November 2019,"['(BBC News)', '(The Guardian)']"


In [3]:
len(df)

21949

In [4]:
df.loc[:, ["text"]].nunique()


text    21864
dtype: int64

In [5]:
# well shit
# okay, lets remove those

# they might possibly be the source of motherfucking data leakage
deduplicated_df = df.drop_duplicates(subset=["text"], keep=False)
len(deduplicated_df)


21779

In [6]:
deduplicated_df.columns

Index(['id', 'title', 'text', 'event_type', 'arguments', 'date', 'metadata'], dtype='object')

In [7]:
deduplicated_df.drop(columns=["id"], inplace=True)

/tmp/ipykernel_11130/2805648092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deduplicated_df.drop(columns=["id"], inplace=True)


In [9]:
deduplicated_df.reset_index(drop=True, inplace=True)
deduplicated_df.index

RangeIndex(start=0, stop=21779, step=1)

In [10]:
deduplicated_df.columns

Index(['title', 'text', 'event_type', 'arguments', 'date', 'metadata'], dtype='object')

In [11]:
deduplicated_df.to_csv("../data/docee/train_all.csv", index_label="id")

In [19]:
from pprint import pformat
from typing import Optional
import os


def deduplicate(
        dataset_path: str,
        subset: Optional[list[str]]=None,
        old_index_name: str="id",
        new_index_name: str="id",
        write_out=print
):
    if subset is None:
        subset = ["text"]

    # load df
    df = pd.read_csv(dataset_path)
    write_out(f"Loaded {len(df)} rows from {os.path.abspath(dataset_path)}")

    # find duplicates
    duplicates: pd.DataFrame = df.loc[df.duplicated(subset=subset, keep=False), :]
    write_out(f"With respect to columns {pformat(subset)}, {len(duplicates)} duplicate examples were found.")

    # group duplicates by label
    # because we only want to drop duplicates which actually have different labels
    group_labels = duplicates.groupby(by=subset).event_type
    write_out(f"Length ")

    # znas sta, doslovno je nebitno sad

    # just remove all the fucking duplicates, not that hard

    pd.read_csv(dataset_path)\
        .drop_duplicates(subset=subset, keep=False)\
        .drop(columns=[old_index_name])\
        .reset_index(drop=True)\
        .to_csv(dataset_path, index_label=new_index_name)


In [20]:
deduplicate("../data/docee/train_all.csv", old_index_name="id", new_index_name="index")

In [21]:
deduplicate("../data/docee/test_all.csv", old_index_name="index", new_index_name="index")

In [22]:
train_path = "../data/docee/train_all.csv"
test_path = "../data/docee/test_all.csv"

print(f"{len(pd.read_csv(train_path)) = }")
print(f"{len(pd.read_csv(test_path)) = }")


len(pd.read_csv(train_path)) = 21779
len(pd.read_csv(test_path)) = 5526


In [17]:
df.loc[:, ["event_type"]].nunique()

event_type    59
dtype: int64

In [18]:
# analyze length of text
train_df = df
test_df = pd.read_csv("../data/docee/test_all.csv")
test_df.head()

,Unnamed: 0,title,text,event_type,arguments,date,metadata
0,0,The anti-Syrian government and the pro-Syrian ...,Rival Lebanese leaders have agreed on steps to...,Sign Agreement,"[{'start': 148, 'end': 206, 'type': 'Contracti...",May 2008,"['(giving it a veto)', '(BBC News)', '(Bloombe..."
1,1,Israel’s PM Benjamin Netanyahu reportedly dela...,The postponement of the military ground offens...,Armed Conflict,"[{'start': 725, 'end': 736, 'type': 'Target', ...",November 2012,"['(RT)', '(BBC)']"
2,2,"A shooting at a billiard hall in El Tarra, Col...",Nine people have been killed in a gun attack i...,Riot,"[{'start': 0, 'end': 27, 'type': 'Casualties a...",July 2018,['(BBC)']
3,3,Under new guidelines to come into force from A...,Private clinics that charge for pregnancy serv...,Government Policy Changes,"[{'start': 0, 'end': 135, 'type': 'Policy Cont...",January 2012,['(BBC)']
4,4,"Newark, New Jersey's 30 public schools shut of...",NEW YORK (Reuters) - Water fountains at 30 sch...,Environment Pollution,"[{'start': 54, 'end': 71, 'type': 'Location', ...",March 2016,['(Reuters)']


In [19]:
from sklearn.model_selection import train_test_split

RANDOM_STATE=18091999

dev_df, test_df = train_test_split(test_df, test_size=0.5, random_state=RANDOM_STATE)

In [20]:
def count_unique_labels(df):
    return df.loc[:, ["event_type"]].nunique()

In [21]:
print(count_unique_labels(train_df))
print(count_unique_labels(dev_df))
print(count_unique_labels(test_df))

event_type    59
dtype: int64
event_type    59
dtype: int64
event_type    59
dtype: int64


In [22]:
from torch.utils.data import Dataset


class DoceeDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        super().__init__()
        self.text = df.text
        self.labels = df.event_type
        self.length = len(self.text)

    def len(self):
        return self.length

    def __getitem__(self, idx: int):
        return self.text[idx], self.labels[idx]


In [23]:
train_dataset = DoceeDataset(train_df)
dev_dataset = DoceeDataset(dev_df)
test_dataset = DoceeDataset(test_df)


In [24]:
text, label = train_dataset[0]
print(f"{text = }, {label = }")

text = 'Vietnam\'s Communist Party Wednesday re-elected its 71-year-old chief for a second term, an expected outcome that sees the conservative pro-China ideologue cementing his hold on power.\nThe party\'s congress elected Nguyen Phu Trong (pronounced Noo-yen Foo Chong) to a 19-member Politburo, the all-powerful body that handles the day-to-day affairs of the government and the party. In a subsequent vote, he was immediately chosen as the general-secretary, the de facto No. 1 leader of the country.\nThe announcement was made on the official Vietnam News Agency\'s website.\nOfficials said Deputy Prime Minister Nguyen Xuan Phuc was also elected to the Politburo, and he is now expected to become the prime minister. He will replace Nguyen Tan Dung, who had had led economic reforms over the last 10 years and had harbored ambitions for the top job. His challenge, however, was snuffed by Trong\'s supporters during the weeklong party congress that ends Thursday.\nThe third most important memb

In [27]:
from transformers import RobertaForSequenceClassification

DEVICE="cuda:0"

model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    cache_dir="../pretrained_models",
    num_labels=59
).to(DEVICE)  # TODO - figure out how was this initialized
print(model)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [26]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base", cache_dir="../pretrained_models")

batch_encoding = tokenizer(
    text,
    truncation=True,
    max_length=512,
    padding=True,
    return_tensors="pt",
)  # so why do we have the option to pad here
print(f"{batch_encoding = }")


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

batch_encoding = {'input_ids': tensor([[    0,   846,  5810,  8697,    18, 12416,  1643,   307,   769,    12,
         15672,    63,  6121,    12,   180,    12,   279,   834,    13,    10,
           200,  1385,     6,    41,   421,  4258,    14,  3681,     5,  3354,
          1759,    12,  8481, 23432, 28789, 10546,   154,    39,   946,    15,
           476,     4, 50118,   133,   537,    18, 12442,  2736, 19211,  4129,
           257,  2393,  1657,    36, 26404, 20155,   440,   139,    12,   219,
           225, 36155, 25932,    43,     7,    10,   753,    12,  8648, 16373,
           428,  7367,     6,     5,    70,    12, 35866,   809,    14, 14617,
             5,   183,    12,   560,    12,  1208,  5185,     9,     5,   168,
             8,     5,   537,     4,    96,    10,  7757,   900,     6,    37,
            21,  1320,  4986,    25,     5,   937,    12, 19301,  1766,     6,
             5,   263, 20624,   440,     4,   112,   884,     9,     5,   247,
             4, 50118

In [29]:
# input_ids, attention_mask
batch_encoding.to(DEVICE)
model_output = model(**batch_encoding)
print(f"{model_output = }")

model_output = SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0779,  0.0104,  0.1149, -0.1154,  0.1039,  0.1422,  0.0217,  0.1132,
          0.0399, -0.0130, -0.1646, -0.1809,  0.1099,  0.1394, -0.1584,  0.1199,
          0.0621,  0.0304,  0.0869, -0.0747,  0.0268,  0.0345,  0.1449,  0.0141,
         -0.2147,  0.1238,  0.1111, -0.0012, -0.0752, -0.0665,  0.0020,  0.0898,
          0.0056,  0.0688,  0.2843,  0.2413,  0.1161,  0.0048, -0.1190, -0.1027,
         -0.0071, -0.0091,  0.1275,  0.0508,  0.1295, -0.0038, -0.0877, -0.1466,
          0.1489, -0.1046, -0.0979, -0.0397, -0.0842,  0.0493,  0.0921,  0.0401,
         -0.0289,  0.0136,  0.1131]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [31]:
from transformers.utils import PaddingStrategy
from transformers import DataCollatorWithPadding

# okay, forward pass works

# next, we need a data collator


data_collator = DataCollatorWithPadding(  # this is obviously not something that we need
    tokenizer=tokenizer,
    padding=PaddingStrategy.LONGEST,
    max_length=tokenizer.model_max_length,
    return_tensors="pt"
)

data_collator_output = data_collator({"text":text})
print(f"{data_collator_output = }")

# sampler -> getitem -> tokenizer -> move_to_device


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['text']

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
)

trainer = Trainer(
    model=model,

)